<a href="https://colab.research.google.com/github/Bahaakhalled/Bahaakhalled.github.io/blob/main/SubmittedMachinfy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align='center'>Unsupervised Textual Similarity</h1>

---

# Business Problem:

Finding the Similarity between multiple long sentences in terms of meaning

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Import Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import nltk
from wordcloud import WordCloud 
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import Pipeline
import spacy
from collections import Counter
nlp = spacy.load('en_core_web_sm')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

###Getting to know the Data

In [3]:
data=pd.read_csv('/content/drive/MyDrive/OZON1.csv', encoding='utf-8')
data.head(10)


,Номер пары,Идентичность пары,Порядковый номер1,ссылка1,название1,цена1,цена по карте1,Описание1,Тип1,Материал1,...,Материал2,Вид игрушки2,"Высота игрушки, см2",Цвет2,Дополнительные функции2,Страна-изготовитель2,Минимальный возраст ребенка2,Персонаж2,Пол ребенка2,Бренд2
0,1,1,1,https://www.ozon.ru/product/myagkaya-igrushka-...,Мягкая игрушка Ам Ням/ Cut the Rope/ Ам ням 20см,430,421.0,Лягушонок Ам Ням из игры Cut the Rope обязател...,Мягкая игрушка,Плюш,...,Плюш,Лягушка,20.0,Зеленый,Без эффектов,Китай,10 месяцев,Ам Ням,Унисекс,NaN
1,2,1,2,https://www.ozon.ru/product/detskaya-myagkaya-...,Детская мягкая игрушка Ам-ням для девочек для ...,392,384.0,"Ам-НямМягкий, нежный друг для ребенка!!!",Мягкая игрушка,Искусственные материалы,...,Плюш,Лягушка,20.0,Зеленый,Без эффектов,Китай,10 месяцев,Ам Ням,Унисекс,NaN
2,3,1,3,https://www.ozon.ru/product/am-nyam-myagkaya-i...,Ам ням мягкая игрушка/Cut the Rope Ам ням/зеле...,419,411.0,Лягушонок Ам Ням из игры Cut the Rope обязател...,Мягкая игрушка,NaN,...,Плюш,Лягушка,20.0,Зеленый,Без эффектов,Китай,10 месяцев,Ам Ням,Унисекс,NaN
3,4,1,4,https://www.ozon.ru/product/myagkaya-igrushka-...,Мягкая игрушка Ам Ням/Ням Плам,419,411.0,Лягушонок Ам Ням из игры Cut the Rope обязател...,Мягкая игрушка,Ткань,...,Плюш,Лягушка,20.0,Зеленый,Без эффектов,Китай,10 месяцев,Ам Ням,Унисекс,NaN
4,5,1,5,https://www.ozon.ru/product/am-nyam-829630932/...,Ам ням,790,774.0,Амням игрушка мягкая необычайно яркая и забавн...,Мягкая игрушка,Плюш,...,Плюш,Лягушка,20.0,Зеленый,Без эффектов,Китай,10 месяцев,Ам Ням,Унисекс,NaN
5,6,0,6,https://www.ozon.ru/product/prosto-toys-cut-th...,"\nPROSTO Toys Cut the Rope, Magic, Om Nom, Ам ...",1315,1289.0,Очаровательный монстрик из серии популярных иг...,Фигурка,"Пластик, ПВХ",...,Плюш,Лягушка,20.0,Зеленый,Без эффектов,Китай,10 месяцев,Ам Ням,Унисекс,NaN
6,7,0,7,https://www.ozon.ru/product/toys-figurka-igrus...,toys Фигурка игрушка башкатряс АмНям персонаж ...,1618,NaN,Фигурка-башкотряс АмНям персонаж популярной ин...,Фигурка,Пластик,...,Плюш,Лягушка,20.0,Зеленый,Без эффектов,Китай,10 месяцев,Ам Ням,Унисекс,NaN
7,8,0,8,https://www.ozon.ru/product/figurka-prosto-toy...,Фигурка PROSTO toys игрушка Ам Ням башкатряс п...,1936,NaN,Фигурка-башкотряс АмНям персонаж популярной ин...,Фигурка,Пластик,...,Плюш,Лягушка,20.0,Зеленый,Без эффектов,Китай,10 месяцев,Ам Ням,Унисекс,NaN
8,9,0,9,https://www.ozon.ru/product/figurka-prosto-toy...,Фигурка PROSTO toys игрушка Ам Ням башкатряс п...,1549,NaN,Фигурка-башкотряс АмНям персонаж популярной ин...,Фигурка,Пластик,...,Плюш,Лягушка,20.0,Зеленый,Без эффектов,Китай,10 месяцев,Ам Ням,Унисекс,NaN
9,10,0,10,https://www.ozon.ru/product/panda-igrushki-943...,Панда игрушки,775,NaN,Панда игрушки,Сквиш,мягкий материал,...,Плюш,Лягушка,20.0,Зеленый,Без эффектов,Китай,10 месяцев,Ам Ням,Унисекс,NaN


In [4]:
copydata=data.copy()
copydata.shape

(121, 36)

In [5]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [6]:
copydata.head(5)

,Номер пары,Идентичность пары,Порядковый номер1,ссылка1,название1,цена1,цена по карте1,Описание1,Тип1,Материал1,Вид игрушки1,"Высота игрушки, см1",Цвет1,Дополнительные функции1,Страна-изготовитель1,Минимальный возраст ребенка1,Персонаж1,Пол ребенка1,Бренд1,Порядковый номер2,ссылка2,название2,цена2,цена по карте2,Описание2,Тип2,Материал2,Вид игрушки2,"Высота игрушки, см2",Цвет2,Дополнительные функции2,Страна-изготовитель2,Минимальный возраст ребенка2,Персонаж2,Пол ребенка2,Бренд2
0,1,1,1,https://www.ozon.ru/product/myagkaya-igrushka-...,Мягкая игрушка Ам Ням/ Cut the Rope/ Ам ням 20см,430,421.0,Лягушонок Ам Ням из игры Cut the Rope обязател...,Мягкая игрушка,Плюш,Лягушка,20.0,Зеленый,Без эффектов,Китай,10 месяцев,Ам Ням,Унисекс,NaN,1,https://www.ozon.ru/product/myagkaya-igrushka-...,Мягкая игрушка Ам Ням/ Cut the Rope/ Ам ням 20см,430,421.0,Лягушонок Ам Ням из игры Cut the Rope обязател...,Мягкая игрушка,Плюш,Лягушка,20.0,Зеленый,Без эффектов,Китай,10 месяцев,Ам Ням,Унисекс,NaN
1,2,1,2,https://www.ozon.ru/product/detskaya-myagkaya-...,Детская мягкая игрушка Ам-ням для девочек для ...,392,384.0,"Ам-НямМягкий, нежный друг для ребенка!!!",Мягкая игрушка,Искусственные материалы,Герои мультфильмов,15.0,Зеленый,Без эффектов,Китай,0 месяцев,NaN,Унисекс,SIDORI,1,https://www.ozon.ru/product/myagkaya-igrushka-...,Мягкая игрушка Ам Ням/ Cut the Rope/ Ам ням 20см,430,421.0,Лягушонок Ам Ням из игры Cut the Rope обязател...,Мягкая игрушка,Плюш,Лягушка,20.0,Зеленый,Без эффектов,Китай,10 месяцев,Ам Ням,Унисекс,NaN
2,3,1,3,https://www.ozon.ru/product/am-nyam-myagkaya-i...,Ам ням мягкая игрушка/Cut the Rope Ам ням/зеле...,419,411.0,Лягушонок Ам Ням из игры Cut the Rope обязател...,Мягкая игрушка,NaN,Лягушка,18.0,NaN,NaN,NaN,От 2 лет,Ам Ням,Унисекс,NaN,1,https://www.ozon.ru/product/myagkaya-igrushka-...,Мягкая игрушка Ам Ням/ Cut the Rope/ Ам ням 20см,430,421.0,Лягушонок Ам Ням из игры Cut the Rope обязател...,Мягкая игрушка,Плюш,Лягушка,20.0,Зеленый,Без эффектов,Китай,10 месяцев,Ам Ням,Унисекс,NaN
3,4,1,4,https://www.ozon.ru/product/myagkaya-igrushka-...,Мягкая игрушка Ам Ням/Ням Плам,419,411.0,Лягушонок Ам Ням из игры Cut the Rope обязател...,Мягкая игрушка,Ткань,Лягушка,18.0,Зеленый,NaN,NaN,10 месяцев,Ам Ням,Унисекс,NaN,1,https://www.ozon.ru/product/myagkaya-igrushka-...,Мягкая игрушка Ам Ням/ Cut the Rope/ Ам ням 20см,430,421.0,Лягушонок Ам Ням из игры Cut the Rope обязател...,Мягкая игрушка,Плюш,Лягушка,20.0,Зеленый,Без эффектов,Китай,10 месяцев,Ам Ням,Унисекс,NaN
4,5,1,5,https://www.ozon.ru/product/am-nyam-829630932/...,Ам ням,790,774.0,Амням игрушка мягкая необычайно яркая и забавн...,Мягкая игрушка,Плюш,Лягушка,20.0,Зеленый,Без эффектов,Китай,От 3 лет,Ам Ням,Унисекс,NaN,1,https://www.ozon.ru/product/myagkaya-igrushka-...,Мягкая игрушка Ам Ням/ Cut the Rope/ Ам ням 20см,430,421.0,Лягушонок Ам Ням из игры Cut the Rope обязател...,Мягкая игрушка,Плюш,Лягушка,20.0,Зеленый,Без эффектов,Китай,10 месяцев,Ам Ням,Унисекс,NaN


**Remove Index and check for missing or duplicates**

In [7]:
copydata[copydata.duplicated()].sum()

Номер пары                      0.0
Идентичность пары               0.0
Порядковый номер1               0.0
ссылка1                         0.0
название1                       0.0
цена1                           0.0
цена по карте1                  0.0
Описание1                       0.0
Тип1                            0.0
Материал1                       0.0
Вид игрушки1                    0.0
Высота игрушки, см1             0.0
Цвет1                           0.0
Дополнительные функции1         0.0
Страна-изготовитель1            0.0
Минимальный возраст ребенка1    0.0
Персонаж1                       0.0
Пол ребенка1                    0.0
Бренд1                          0.0
Порядковый номер2               0.0
ссылка2                         0.0
название2                       0.0
цена2                           0.0
цена по карте2                  0.0
Описание2                       0.0
Тип2                            0.0
Материал2                       0.0
Вид игрушки2                

In [8]:
copydata.info()
# Displays colum names, complete (non-missing) cases per colum, and datatype per colum

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 36 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Номер пары                    121 non-null    int64  
 1   Идентичность пары             121 non-null    int64  
 2   Порядковый номер1             121 non-null    int64  
 3   ссылка1                       121 non-null    object 
 4   название1                     121 non-null    object 
 5   цена1                         121 non-null    int64  
 6   цена по карте1                77 non-null     float64
 7   Описание1                     121 non-null    object 
 8   Тип1                          121 non-null    object 
 9   Материал1                     110 non-null    object 
 10  Вид игрушки1                  121 non-null    object 
 11  Высота игрушки, см1           121 non-null    float64
 12  Цвет1                         110 non-null    object 
 13  Допол

Check for missing values and clean


In [9]:
copydata.isnull()
# It returns 'False' if a value is not missing and 'True' if a value is missing, for each value in a dataframe

,Номер пары,Идентичность пары,Порядковый номер1,ссылка1,название1,цена1,цена по карте1,Описание1,Тип1,Материал1,Вид игрушки1,"Высота игрушки, см1",Цвет1,Дополнительные функции1,Страна-изготовитель1,Минимальный возраст ребенка1,Персонаж1,Пол ребенка1,Бренд1,Порядковый номер2,ссылка2,название2,цена2,цена по карте2,Описание2,Тип2,Материал2,Вид игрушки2,"Высота игрушки, см2",Цвет2,Дополнительные функции2,Страна-изготовитель2,Минимальный возраст ребенка2,Персонаж2,Пол ребенка2,Бренд2
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
2,False,False,False,False,False,False,False,False,False,True,False,False,True,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
5,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
6,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
7,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
8,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
9,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True


In [10]:
copydata.isnull().sum()
#Показывает количество отсутствующих значений по каждому столбцу

Номер пары                       0
Идентичность пары                0
Порядковый номер1                0
ссылка1                          0
название1                        0
цена1                            0
цена по карте1                  44
Описание1                        0
Тип1                             0
Материал1                       11
Вид игрушки1                     0
Высота игрушки, см1              0
Цвет1                           11
Дополнительные функции1         33
Страна-изготовитель1            44
Минимальный возраст ребенка1     0
Персонаж1                       22
Пол ребенка1                     0
Бренд1                          55
Порядковый номер2                0
ссылка2                          0
название2                        0
цена2                            0
цена по карте2                  44
Описание2                        0
Тип2                             0
Материал2                       11
Вид игрушки2                     0
Высота игрушки, см2 

In [11]:
copydata['цена по карте1'].fillna(copydata['цена1'], inplace=True)
#Заменяем отсутствующие значения в столбце 'цена по карте1' значениями из столбца 'цена1'

In [12]:
copydata.iloc[6]

Номер пары                                                                      7
Идентичность пары                                                               0
Порядковый номер1                                                               7
ссылка1                         https://www.ozon.ru/product/toys-figurka-igrus...
название1                       toys Фигурка игрушка башкатряс АмНям персонаж ...
цена1                                                                        1618
цена по карте1                                                             1618.0
Описание1                       Фигурка-башкотряс АмНям персонаж популярной ин...
Тип1                                                                      Фигурка
Материал1                                                                 Пластик
Вид игрушки1                                                     \nФункциональная
Высота игрушки, см1                                                          10.0
Цвет1           

In [13]:
copydata['цена по карте1'].isnull().sum()

0

In [14]:
copydata['Материал1'].unique()

array(['Плюш', 'Искусственные материалы', nan, 'Ткань', 'Пластик, ПВХ',
       'Пластик', 'мягкий материал'], dtype=object)

Учитывая, что отсутствует единый стандарт для заполнения поля имеет смысл сравнивать поле по тексту, а не создавать отдельные категории. Либо все редкие категории объединить в одну

In [15]:

copydata['Материал1'].fillna('МатериалНеИзвестен', inplace=True)
# Заменяем пустые значения на "материалнеизвестен" либо можно заменить на наиболее часто встречающиеся значения

In [16]:
copydata['Материал1'].isnull().sum()

0

In [17]:
copydata['Цвет1'].unique()

array(['Зеленый', nan, 'Белый, черный', 'Зеленый, белый'], dtype=object)

In [18]:
copydata['Цвет1'].fillna(copydata['Цвет1'].mode(), inplace=True)
# Заменяем пустые значения на наиболее часто встречающиеся


In [19]:
copydata['Цвет1'].isnull().sum()

11

In [20]:
copydata['Цвет1'].mode()

0           Зеленый
1    Зеленый, белый
Name: Цвет1, dtype: object

In [21]:
copydata['Цвет1'].fillna(copydata['Цвет1'].mode()[0],inplace=True)
# заменили пустые значения на 'Зеленый'

In [22]:
copydata['Цвет1']. isnull().sum()

0

In [23]:
copydata['Страна-изготовитель1'].unique()

array(['Китай', nan, 'Россия'], dtype=object)

In [24]:
# Ввиду того, что отсутствует 30% значений, для отсутствующих значений отражаем 'Страна_изготовитель_не_известна'
copydata['Страна-изготовитель1'].fillna('CтранаИзготовительНеИзвестна', inplace=True)

In [25]:
copydata['Страна-изготовитель1'].isnull().sum()

0

In [26]:
copydata['Персонаж1'].fillna('ПерсонажНеИзвестен', inplace = True)
# отсутствует 20% значений, заменяем на "Персонаж_не_известен"

In [27]:
copydata['Персонаж1'].isnull().sum()

0

In [28]:
copydata['Бренд1'].fillna('БрендНеИзвестен', inplace = True)
# отсутствующие значения заменяем на 'Бренд_не_известен'

In [29]:
copydata['Бренд1'].isnull().sum()

0

In [30]:
copydata['цена по карте2'].fillna(copydata['цена2'], inplace=True)
#Заменяем отсутствующие значения в столбце 'цена по карте2' значениями из столбца 'цена2'

In [31]:
copydata['цена по карте2'].isnull().sum()

0

In [32]:
copydata['Материал2'].fillna('МатериалНеИзвестен', inplace=True)

In [33]:
copydata['Материал2'].isnull().sum()

0

In [34]:
copydata['Цвет2'].fillna(copydata['Цвет2'].mode()[0],inplace=True)

In [35]:
copydata['Страна-изготовитель2'].fillna('CтранаИзготовительНеИзвестна', inplace=True)

In [36]:
copydata['Персонаж2'].fillna('ПерсонажНеИзвестен', inplace = True)

In [37]:
copydata['Бренд2']=data['Бренд2']
copydata['Бренд2'].fillna('БрендНеИзвестен', inplace = True)

In [38]:
copydata.isnull().sum()

Номер пары                       0
Идентичность пары                0
Порядковый номер1                0
ссылка1                          0
название1                        0
цена1                            0
цена по карте1                   0
Описание1                        0
Тип1                             0
Материал1                        0
Вид игрушки1                     0
Высота игрушки, см1              0
Цвет1                            0
Дополнительные функции1         33
Страна-изготовитель1             0
Минимальный возраст ребенка1     0
Персонаж1                        0
Пол ребенка1                     0
Бренд1                           0
Порядковый номер2                0
ссылка2                          0
название2                        0
цена2                            0
цена по карте2                   0
Описание2                        0
Тип2                             0
Материал2                        0
Вид игрушки2                     0
Высота игрушки, см2 

In [39]:

copydata['Дополнительные функции1'].fillna('ДополнительныеФункцииНеИзвестны', inplace=True)

In [40]:
copydata['Дополнительные функции2'].fillna('ДополнительныеФункцииНеИзвестны', inplace=True)

In [41]:
copydata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 36 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Номер пары                    121 non-null    int64  
 1   Идентичность пары             121 non-null    int64  
 2   Порядковый номер1             121 non-null    int64  
 3   ссылка1                       121 non-null    object 
 4   название1                     121 non-null    object 
 5   цена1                         121 non-null    int64  
 6   цена по карте1                121 non-null    float64
 7   Описание1                     121 non-null    object 
 8   Тип1                          121 non-null    object 
 9   Материал1                     121 non-null    object 
 10  Вид игрушки1                  121 non-null    object 
 11  Высота игрушки, см1           121 non-null    float64
 12  Цвет1                         121 non-null    object 
 13  Допол

In [42]:
copydata['цена по карте1']=pd.to_numeric(copydata['цена по карте1'], downcast='signed')


In [43]:
copydata['цена по карте1'].unique()

array([ 421,  384,  411,  774, 1289, 1618, 1936, 1549,  775, 1645],
      dtype=int16)

In [44]:
data['цена по карте1'].unique()

array([ 421.,  384.,  411.,  774., 1289.,   nan, 1645.])

In [45]:
copydata['цена по карте1'].info()
# Надо, наверное, перевести в int64. Не искала как делать.

<class 'pandas.core.series.Series'>
RangeIndex: 121 entries, 0 to 120
Series name: цена по карте1
Non-Null Count  Dtype
--------------  -----
121 non-null    int16
dtypes: int16(1)
memory usage: 370.0 bytes


In [46]:
copydata['цена по карте2']=pd.to_numeric(copydata['цена по карте2'], downcast='signed')

In [47]:
copydata['цена по карте2'].unique()

array([ 421,  384,  411,  774, 1289, 1618, 1936, 1549,  775, 1645],
      dtype=int16)

In [48]:
data['цена по карте2'].unique()

array([ 421.,  384.,  411.,  774., 1289.,   nan, 1645.])

In [49]:
copydata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 36 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Номер пары                    121 non-null    int64  
 1   Идентичность пары             121 non-null    int64  
 2   Порядковый номер1             121 non-null    int64  
 3   ссылка1                       121 non-null    object 
 4   название1                     121 non-null    object 
 5   цена1                         121 non-null    int64  
 6   цена по карте1                121 non-null    int16  
 7   Описание1                     121 non-null    object 
 8   Тип1                          121 non-null    object 
 9   Материал1                     121 non-null    object 
 10  Вид игрушки1                  121 non-null    object 
 11  Высота игрушки, см1           121 non-null    float64
 12  Цвет1                         121 non-null    object 
 13  Допол

In [50]:
copydata['Высота игрушки, см1'].unique()

array([20. , 15. , 18. ,  3.5, 10. ])

In [51]:
copydata['Высота игрушки, см2'].unique()


array([20. , 15. , 18. ,  3.5, 10. ])

In [52]:
copydata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 36 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Номер пары                    121 non-null    int64  
 1   Идентичность пары             121 non-null    int64  
 2   Порядковый номер1             121 non-null    int64  
 3   ссылка1                       121 non-null    object 
 4   название1                     121 non-null    object 
 5   цена1                         121 non-null    int64  
 6   цена по карте1                121 non-null    int16  
 7   Описание1                     121 non-null    object 
 8   Тип1                          121 non-null    object 
 9   Материал1                     121 non-null    object 
 10  Вид игрушки1                  121 non-null    object 
 11  Высота игрушки, см1           121 non-null    float64
 12  Цвет1                         121 non-null    object 
 13  Допол

#Создание новых столбцов


In [53]:
#Создаем новые столбцы:
#- Разница между столбцами цена1 и цена2.
#- Разница между столбцами цена по карте1 и цена по карте2
#- Разница между столбцами Высота игрушки, см1 и Высота игрушки, см2

copydata['Разница между столбцами "цена1" и "цена2"'] =  copydata['цена1'] - copydata['цена2']


In [54]:
copydata['Разница между столбцами "цена по карте1" и "цена по карте2"'] = copydata['цена по карте1'] - copydata['цена по карте2']


In [55]:

copydata['Разница между столбцами "Высота игрушки, см1" и "Высота игрушки, см2"'] = copydata['Высота игрушки, см1'] - copydata['Высота игрушки, см2']


In [56]:
copydata['Разница между столбцами "цена1" и "цена2"'].unique()

array([    0,   -38,   -11,   360,   885,  1188,  1506,  1119,   345,
        3070,    38,    27,   398,   923,  1226,  1544,  1157,   383,
        3108,    11,   -27,   371,   896,  1199,  1517,  1130,   356,
        3081,  -360,  -398,  -371,   525,   828,  1146,   759,   -15,
        2710,  -885,  -923,  -896,  -525,   303,   621,   234,  -540,
        2185, -1188, -1226, -1199,  -828,  -303,   318,   -69,  -843,
        1882, -1506, -1544, -1517, -1146,  -621,  -318,  -387, -1161,
        1564, -1119, -1157, -1130,  -759,  -234,    69,   387,  -774,
        1951,  -345,  -383,  -356,    15,   540,   843,  1161,   774,
        2725, -3070, -3108, -3081, -2710, -2185, -1882, -1564, -1951,
       -2725])

In [57]:
copydata['Разница между столбцами "цена по карте1" и "цена по карте2"'].unique()


array([    0,   -37,   -10,   353,   868,  1197,  1515,  1128,   354,
        1224,    37,    27,   390,   905,  1234,  1552,  1165,   391,
        1261,    10,   -27,   363,   878,  1207,  1525,  1138,   364,
        -353,  -390,  -363,   515,   844,  1162,   775,     1,   871,
        -868,  -905,  -878,  -515,   329,   647,   260,  -514,   356,
       -1197, -1234, -1207,  -844,  -329,   318,   -69,  -843, -1515,
       -1552, -1525, -1162,  -647,  -318,  -387, -1161,  -291, -1128,
       -1165, -1138,  -775,  -260,    69,   387,  -774,    96,  -354,
        -391,  -364,    -1,   514,   843,  1161,   774,   870, -1224,
       -1261,  -871,  -356,   291,   -96,  -870], dtype=int16)

In [58]:
copydata['Разница между столбцами "Высота игрушки, см1" и "Высота игрушки, см2"'].unique()

array([  0. ,  -5. ,  -2. , -16.5, -10. ,   5. ,   3. , -11.5,   2. ,
        -3. , -14.5,  -8. ,  16.5,  11.5,  14.5,   6.5,  10. ,   8. ,
        -6.5])

In [59]:
copydata['Разница между столбцами "цена1" и "цена2"'] = abs(copydata['Разница между столбцами "цена1" и "цена2"'])

In [60]:
copydata['Разница между столбцами "цена1" и "цена2"'].unique()

array([   0,   38,   11,  360,  885, 1188, 1506, 1119,  345, 3070,   27,
        398,  923, 1226, 1544, 1157,  383, 3108,  371,  896, 1199, 1517,
       1130,  356, 3081,  525,  828, 1146,  759,   15, 2710,  303,  621,
        234,  540, 2185,  318,   69,  843, 1882,  387, 1161, 1564,  774,
       1951, 2725])

In [61]:
copydata['Разница между столбцами "Высота игрушки, см1" и "Высота игрушки, см2"'] = abs(copydata['Разница между столбцами "Высота игрушки, см1" и "Высота игрушки, см2"'])

In [62]:
copydata['Разница между столбцами "цена по карте1" и "цена по карте2"'] = abs(copydata['Разница между столбцами "цена по карте1" и "цена по карте2"'])


In [63]:
# создание столбцов с относительной разницей к цене и высоте игрушки первого товара
copydata['Относительная разница в цене'] = copydata['Разница между столбцами "цена1" и "цена2"']/copydata['цена1']
copydata['Относительная разница в цене по карте'] = copydata['Разница между столбцами "цена по карте1" и "цена по карте2"']/copydata['цена по карте1']
copydata['Относительная разница в высоте игрушки'] = copydata['Разница между столбцами "Высота игрушки, см1" и "Высота игрушки, см2"']/copydata['Высота игрушки, см1']

In [64]:
copydata['Относительная разница в цене'].head(10)

0    0.000000
1    0.096939
2    0.026253
3    0.026253
4    0.455696
5    0.673004
6    0.734240
7    0.777893
8    0.722402
9    0.445161
Name: Относительная разница в цене, dtype: float64

In [65]:
copydata['Относительная разница в цене по карте'].head(10)

0    0.000000
1    0.096354
2    0.024331
3    0.024331
4    0.456072
5    0.673390
6    0.739802
7    0.782541
8    0.728212
9    0.456774
Name: Относительная разница в цене по карте, dtype: float64

In [66]:
copydata['Относительная разница в высоте игрушки'].head(10)

0    0.000000
1    0.333333
2    0.111111
3    0.111111
4    0.000000
5    4.714286
6    1.000000
7    1.000000
8    1.000000
9    0.333333
Name: Относительная разница в высоте игрушки, dtype: float64

In [67]:
copydata.head(5)

,Номер пары,Идентичность пары,Порядковый номер1,ссылка1,название1,цена1,цена по карте1,Описание1,Тип1,Материал1,Вид игрушки1,"Высота игрушки, см1",Цвет1,Дополнительные функции1,Страна-изготовитель1,Минимальный возраст ребенка1,Персонаж1,Пол ребенка1,Бренд1,Порядковый номер2,ссылка2,название2,цена2,цена по карте2,Описание2,Тип2,Материал2,Вид игрушки2,"Высота игрушки, см2",Цвет2,Дополнительные функции2,Страна-изготовитель2,Минимальный возраст ребенка2,Персонаж2,Пол ребенка2,Бренд2,"Разница между столбцами ""цена1"" и ""цена2""","Разница между столбцами ""цена по карте1"" и ""цена по карте2""","Разница между столбцами ""Высота игрушки, см1"" и ""Высота игрушки, см2""",Относительная разница в цене,Относительная разница в цене по карте,Относительная разница в высоте игрушки
0,1,1,1,https://www.ozon.ru/product/myagkaya-igrushka-...,Мягкая игрушка Ам Ням/ Cut the Rope/ Ам ням 20см,430,421,Лягушонок Ам Ням из игры Cut the Rope обязател...,Мягкая игрушка,Плюш,Лягушка,20.0,Зеленый,Без эффектов,Китай,10 месяцев,Ам Ням,Унисекс,БрендНеИзвестен,1,https://www.ozon.ru/product/myagkaya-igrushka-...,Мягкая игрушка Ам Ням/ Cut the Rope/ Ам ням 20см,430,421,Лягушонок Ам Ням из игры Cut the Rope обязател...,Мягкая игрушка,Плюш,Лягушка,20.0,Зеленый,Без эффектов,Китай,10 месяцев,Ам Ням,Унисекс,БрендНеИзвестен,0,0,0.0,0.000000,0.000000,0.000000
1,2,1,2,https://www.ozon.ru/product/detskaya-myagkaya-...,Детская мягкая игрушка Ам-ням для девочек для ...,392,384,"Ам-НямМягкий, нежный друг для ребенка!!!",Мягкая игрушка,Искусственные материалы,Герои мультфильмов,15.0,Зеленый,Без эффектов,Китай,0 месяцев,ПерсонажНеИзвестен,Унисекс,SIDORI,1,https://www.ozon.ru/product/myagkaya-igrushka-...,Мягкая игрушка Ам Ням/ Cut the Rope/ Ам ням 20см,430,421,Лягушонок Ам Ням из игры Cut the Rope обязател...,Мягкая игрушка,Плюш,Лягушка,20.0,Зеленый,Без эффектов,Китай,10 месяцев,Ам Ням,Унисекс,БрендНеИзвестен,38,37,5.0,0.096939,0.096354,0.333333
2,3,1,3,https://www.ozon.ru/product/am-nyam-myagkaya-i...,Ам ням мягкая игрушка/Cut the Rope Ам ням/зеле...,419,411,Лягушонок Ам Ням из игры Cut the Rope обязател...,Мягкая игрушка,МатериалНеИзвестен,Лягушка,18.0,Зеленый,ДополнительныеФункцииНеИзвестны,CтранаИзготовительНеИзвестна,От 2 лет,Ам Ням,Унисекс,БрендНеИзвестен,1,https://www.ozon.ru/product/myagkaya-igrushka-...,Мягкая игрушка Ам Ням/ Cut the Rope/ Ам ням 20см,430,421,Лягушонок Ам Ням из игры Cut the Rope обязател...,Мягкая игрушка,Плюш,Лягушка,20.0,Зеленый,Без эффектов,Китай,10 месяцев,Ам Ням,Унисекс,БрендНеИзвестен,11,10,2.0,0.026253,0.024331,0.111111
3,4,1,4,https://www.ozon.ru/product/myagkaya-igrushka-...,Мягкая игрушка Ам Ням/Ням Плам,419,411,Лягушонок Ам Ням из игры Cut the Rope обязател...,Мягкая игрушка,Ткань,Лягушка,18.0,Зеленый,ДополнительныеФункцииНеИзвестны,CтранаИзготовительНеИзвестна,10 месяцев,Ам Ням,Унисекс,БрендНеИзвестен,1,https://www.ozon.ru/product/myagkaya-igrushka-...,Мягкая игрушка Ам Ням/ Cut the Rope/ Ам ням 20см,430,421,Лягушонок Ам Ням из игры Cut the Rope обязател...,Мягкая игрушка,Плюш,Лягушка,20.0,Зеленый,Без эффектов,Китай,10 месяцев,Ам Ням,Унисекс,БрендНеИзвестен,11,10,2.0,0.026253,0.024331,0.111111
4,5,1,5,https://www.ozon.ru/product/am-nyam-829630932/...,Ам ням,790,774,Амням игрушка мягкая необычайно яркая и забавн...,Мягкая игрушка,Плюш,Лягушка,20.0,Зеленый,Без эффектов,Китай,От 3 лет,Ам Ням,Унисекс,БрендНеИзвестен,1,https://www.ozon.ru/product/myagkaya-igrushka-...,Мягкая игрушка Ам Ням/ Cut the Rope/ Ам ням 20см,430,421,Лягушонок Ам Ням из игры Cut the Rope обязател...,Мягкая игрушка,Плюш,Лягушка,20.0,Зеленый,Без эффектов,Китай,10 месяцев,Ам Ням,Унисекс,БрендНеИзвестен,360,353,0.0,0.455696,0.456072,0.000000


In [68]:
#Объединяем в одно значение характеристики, имеющие менее 10% отсутствующих значений и отдельно более 10% (экспертно) без описания
copydata['Характеристики отсутствующие значения <10%_1'] = copydata['Тип1'] + ' '  + copydata['Материал1'] + ' '\
+ copydata['Вид игрушки1']  + ' ' + copydata['Цвет1'] + ' '  + copydata['Минимальный возраст ребенка1']  + ' ' + copydata['Пол ребенка1']


In [69]:
copydata['Характеристики отсутствующие значения <10%_2'] = copydata['Тип2'] + ' '+ copydata['Материал2']+ ' ' + copydata['Вид игрушки2']\
+ ' ' + copydata['Цвет2']+ ' ' + copydata['Минимальный возраст ребенка2'] + ' '+ copydata['Пол ребенка2']

In [70]:
copydata['Характеристики отсутствующие значения >10%_1'] = copydata['Дополнительные функции1']+ ' ' + copydata['Страна-изготовитель1']+ ' ' + copydata['Персонаж1'] + ' '+ copydata['Бренд1'] 

In [71]:
copydata['Характеристики отсутствующие значения >10%_2'] = copydata['Дополнительные функции2']+ ' ' + copydata['Страна-изготовитель2']+ ' ' + copydata['Персонаж2'] + ' '+ copydata['Бренд2'] 

In [72]:
copydata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 46 columns):
 #   Column                                                                 Non-Null Count  Dtype  
---  ------                                                                 --------------  -----  
 0   Номер пары                                                             121 non-null    int64  
 1   Идентичность пары                                                      121 non-null    int64  
 2   Порядковый номер1                                                      121 non-null    int64  
 3   ссылка1                                                                121 non-null    object 
 4   название1                                                              121 non-null    object 
 5   цена1                                                                  121 non-null    int64  
 6   цена по карте1                                                         121 non-null    int

In [74]:
# dataframe с новыми столбцами без пропущенных значений
copydata.to_csv('data-preparation-1-result.csv')
